<a href="https://colab.research.google.com/github/robertjprior/CausaLM/blob/master/Mistral_7B_Instruct_Summarization_Tests_PRE_TRAINING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os, sys
from pathlib import Path
from google.colab import drive

nb_path = 'generated_training_data'
try:
  drive.mount('/content/drive')
  #mounting: https://stackoverflow.com/questions/55253498/how-do-i-install-a-library-permanently-in-colab

  os.symlink('/content/drive/MyDrive/Colab Notebooks/MistralSummarization', nb_path)

  sys.path.insert(0,nb_path)
except FileExistsError:
  pass

Mounted at /content/drive


In [2]:
# import torch
# if str(torch.__version__) != "2.1.1+cu121"
#   print(torch.__version__)
# # 1.13.0+cu116

#   !pip3 install numpy --pre torch[dynamo] --force-reinstall --extra-index-url https://download.pytorch.org/whl/nightly/cu117


#   # Click on RESTART RUNTIME

# import torch
# print(torch.__version__)
# # 1.14.0.dev20221207+cu117

In [3]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip install -q datasets loralib sentencepiece
!pip -q install bitsandbytes xformers einops
!pip -q install langchain
!pip install git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git



  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 MB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.6 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/peft.git to /tmp

In [4]:
#!pip install transformers==4.30

In [5]:
from datasets import Dataset
from pathlib import Path
import datasets

In [6]:
!nvidia-smi

Tue Nov 21 18:06:40 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    26W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Mistral 7B Instruct



In [7]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [8]:
from peft import PeftModel, PeftConfig
checkpoint = str(Path(nb_path, "checkpoint-217"))
peft_config = PeftConfig.from_pretrained(checkpoint)

In [9]:
#torch.set_default_device('cuda')

In [10]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    #load_in_8bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [11]:

tokenizer = AutoTokenizer.from_pretrained(peft_config.base_model_name_or_path)

model = AutoModelForCausalLM.from_pretrained(peft_config.base_model_name_or_path, quantization_config=bnb_config, trust_remote_code=True, device_map='auto')
                                          #torch_dtype="auto")



tokenizer_config.json:   0%|          | 0.00/963 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

...

pytorch_model-00006-of-00008.bin:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

pytorch_model-00007-of-00008.bin:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

pytorch_model-00008-of-00008.bin:   0%|          | 0.00/816M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [12]:
# text = "<s>[INST] What is your favourite condiment? [/INST]Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!</s> [INST] Do you have mayonnaise recipes? [/INST]"

# encodeds = tokenizer(text, return_tensors="pt", add_special_tokens=False)

# device = 'cuda'
# model_inputs = encodeds.to(device)
# #model.to(device)

# generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True)
# decoded = tokenizer.batch_decode(generated_ids)
# print(decoded[0])

In [13]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRM

In [14]:
#turn this off for pretraining evaluation
#model = PeftModel.from_pretrained(model, checkpoint)
#model = model.merge_and_unload()

In [15]:
model.eval()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRM

In [16]:
# tokenizer.pad_token_id = tokenizer.eos_token_id
# tokenizer.pad_token = tokenizer.eos_token
# model.config.pad_token_id = model.config.eos_token_id

# Prep Summarization Dataset for training
dataset: https://github.com/Alex-Fabbri/Multi-News

In [17]:
training_dataset_size = 200

In [18]:
file = open(Path(nb_path, 'test.txt.src.tokenized.fixed.cleaned.final.truncated')).read()
articles = file.split('\n')[0:training_dataset_size]
file = open(Path(nb_path, 'test.txt.tgt.tokenized.fixed.cleaned.final.truncated')).read()
target = file.split('\n')[0:training_dataset_size]

print(len(articles))
print(len(target))

200
200


In [19]:
def formatting_prompts_func_batch(articles, summary):
    output_texts = []
    for i in range(len(articles)):
        #text = f"### Question: {example['prompt'][i]}\n ### Answer: {example['response'][i]}"
        text = f"<s>[INST] Provide a concise summary of the triple backquoted text. '''{articles[i]}'''  Provide the summary in bullet points. [/INST] \\n {summary[i]} </s>"
        output_texts.append(text)
    return output_texts

In [20]:
train_qa = formatting_prompts_func_batch(articles, target)
train_qa = {"quote":train_qa}

#test_qa = formatting_prompts_func(test_df)
#test_qa = {"quote":test_qa}

In [21]:
train_qa = Dataset.from_dict(train_qa)
train_qa = train_qa.map(lambda samples: tokenizer(samples["quote"], truncation=True), batched=True)

#test_qa = Dataset.from_dict(test_qa)
#test_qa = test_qa.map(lambda samples: tokenizer(samples["quote"]), batched=True)
train_qa

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Dataset({
    features: ['quote', 'input_ids', 'attention_mask'],
    num_rows: 200
})

In [22]:

data = datasets.DatasetDict({"train":train_qa})#, "test": test_qa})

# 5 Levels of Summarization with Mistral

## Level 1 - a couple of sentence

In [23]:
def formatting_prompts_func(example):
    text = f"<s>[INST] Provide a concise summary of the triple backquoted text. '''{example}'''  Provide the summary in bullet points. [/INST] \\n "
    return text

In [24]:
formatting_prompts_func("hi how are you doing today?")

"<s>[INST] Provide a concise summary of the triple backquoted text. '''hi how are you doing today?'''  Provide the summary in bullet points. [/INST] \\n "

In [25]:
from transformers import GenerationConfig
def generate(instruction, input=None):
    #prompt = generate_prompt(instruction, input)
    instruction = formatting_prompts_func(instruction)
    inputs = tokenizer(instruction, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    print("num tokens: ")
    print(input_ids.shape[1])
    generation_output = model.generate(
        input_ids=input_ids,
        do_sample=True,
        generation_config=GenerationConfig(num_beams=4, top_p=0.2, do_sample=True),
        #generation_config=GenerationConfig(temperature=0.2, num_beams=4, top_p=0.2, do_sample=True),
        return_dict_in_generate=True,
        output_scores=True,
        no_repeat_ngram_size=3,
        early_stopping=True,
        max_new_tokens=200,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id = tokenizer.eos_token_id
    )
    for seq in generation_output.sequences:
        output = tokenizer.decode(seq)
        #print(output)
        #print(output.replace(instruction, "PROMPT PLACEHOLDER"))
    return output

In [26]:
text = """ Philosophy (from Greek: φιλοσοφία, philosophia, 'love of wisdom') \
is the systematized study of general and fundamental questions, \
such as those about existence, reason, knowledge, values, mind, and language. \
Some sources claim the term was coined by Pythagoras (c. 570 – c. 495 BCE), \
although this theory is disputed by some. Philosophical methods include questioning, \
critical discussion, rational argument, and systematic presentation.
"""

In [27]:
# encodeds = tokenizer(text, return_tensors="pt", add_special_tokens=False)

# device = 'cuda'
# model_inputs = encodeds.to(device)
# #model.to(device)

# generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True)
# decoded = tokenizer.batch_decode(generated_ids)
# print(decoded[0])

In [28]:
#prompt = formatting_prompts_func(text)
generate(text)

num tokens: 
152


"<s><s>[INST] Provide a concise summary of the triple backquoted text. ''' Philosophy (from Greek: φιλοσοφία, philosophia, 'love of wisdom') is the systematized study of general and fundamental questions, such as those about existence, reason, knowledge, values, mind, and language. Some sources claim the term was coined by Pythagoras (c.\u2009570 – c.\u2009495 BCE), although this theory is disputed by some. Philosophical methods include questioning, critical discussion, rational argument, and systematic presentation.\n'''  Provide the summary in bullet points. [/INST] \\n  \\n\n\n- Philosophy is a discipline that deals with the fundamental nature of knowledge, reality, and existence, especially when considered as an academic discipline.\n- It is distinguished from other ways of addressing fundamental questions by its critical, generally systematic approach and its reliance on rational argument when analyzing, probing, and resolving doctrines. \\n \\n</s>"

## Level 2 - summarize a couple paragraphs

In [29]:
essay_1 = """The 2023 Israel–Hamas war is an ongoing armed conflict between Israel and Hamas-led Palestinian militant groups taking place chiefly in and around the Gaza Strip, with clashes also taking place in the West Bank and Israel-Lebanon border. On 7 October 2023, Hamas[o]-led Palestinian militants[p] launched a multi-pronged invasion of southern Israel from the Gaza Strip.[48] The surprise attack comprised a barrage of rockets, while around 3,000 militants breached the Gaza–Israel barrier and attacked Israeli military bases and civilian population centres, as well as a music festival near Re'im. More than 800 Israeli civilians and more than 200 soldiers were killed during the invasion and an estimated 240[49] Israeli and foreign nationals were taken as captives or hostages to the Gaza Strip. The attack has been described as an act of terror by 44 nations,[50] while Hamas officials stated their goal was to liberate their Israeli-occupied land, their holy sites and their prisoners in Israel.[51][52][53] After clearing Hamas militants, the Israeli military retaliated by conducting an extensive aerial bombardment campaign in which 6,000 bombs were dropped on Gazan targets[54] over six days and by imposing a total blockade of the Gaza strip followed by a large-scale ground invasion of Gaza.[55][56]

Israel's declaration of a state of war marked the start of the most significant military escalation in the region since the Yom Kippur War in 1973. As of 14 November 2023 more than 12,000 Palestinians, including 5,000 children, have been killed, making this the deadliest war for children in modern times.[57][17][58][59] UN Secretary General António Guterres said Gaza is "becoming a graveyard for children".[60] Widespread civilian deaths have led to both Israel and Hamas being accused of war crimes.[61][62] Israel urged Gazans to evacuate northern Gaza, while Hamas called on residents to stay in their homes.[63][62][64] The United Nations reported that around 1.5 million Palestinians, over 70% of Gaza's population,[46] as well as over 200,000 Israelis have been internally displaced.[65] The war has led to a severe humanitarian crisis in Gaza. The health system is on the brink of collapse, most hospitals are out of service,[66] and there are acute shortages of drinking water, food, fuel and medical supplies such as anesthetics for C-sections and amputations.[67][68] The United Nations has warned of the "immediate possibility" of starvation and the spread of disease in the region.[60][69][70]

The war has sparked widespread global protests that have focused on a variety of issues including demands for a ceasefire and the release of hostages. The United States vetoed a United Nations Security Council resolution calling for an immediate humanitarian ceasefire;[71] a week later, a non-binding advisory resolution passed overwhelmingly in the United Nations General Assembly.[72] Israel has rejected ceasefire calls, citing the release of all hostages taken by a Hamas as a precondition for a ceasefire.[73] In the early weeks of conflict, Hamas had announced that it would release all Israeli hostages in exchange for Israel's release of all Palestinian prisoners.[74] On 9 November, Israel agreed to daily four-hour "humanitarian pauses" to allow civilians to obtain food and medicine, as well as evacuate to the south.[75] On 15 November, the UN Security Council approved a resolution calling for "urgent and extended humanitarian pauses and corridors throughout the Gaza Strip."[76] Israel's ambassador to the UN said that they would refuse to abide by the resolution.[76]

Background
See also: Israeli–Palestinian conflict and Gaza–Israel conflict
The Gaza Strip and Israel have been in conflict since the Israeli withdrawal from Gaza in 2005, which it had hitherto occupied since 1967. The United Nations and several human rights organizations continue to classify Gaza as held under Israeli occupation, due to its effective military control over the territory.[77][78]

Hamas's victory in the 2006 Palestinian legislative election and the subsequent battle between it and Fatah, which Hamas ultimately won, significantly escalated tensions with Israel.[79][80] Israel, along with Egypt, imposed a blockade that significantly damaged Gaza's economy, citing security concerns as the justification.[81] International rights groups have characterized the blockade as a form of collective punishment,[82] while Israel defended it as necessary to prevent weapons and dual-use goods from entering the territory.[83][84] According to the United Nations Office for the Coordination of Humanitarian Affairs (UNOCHA), Palestinian militant groups fired ~2,700 rockets into Israel from September 2005 through May 2007.[85]

Since the blockade, Israel and Palestinian militants have had several clashes and made attacks on each other.[81][86][87] The Palestinian Authority has not held national elections since 2006.[81][88] Hamas tunneled under the border wall to launch cross-border attacks and fired rockets into Israeli territory. This led to multiple conflicts, escalating into outright wars. Israel relied on the Iron Dome rocket defense system for defense, and responded with targeted strikes into Gaza, aiming to minimize the militant threat.[87] Surveys in 2023 of Palestinians in Gaza and the West Bank before the war indicated that a majority supported the use of "armed struggle", the creation of "militant groups", and an intifada ("uprising") against the Israeli occupation.[89][90]

Hamas leadership
Further information: Governance of the Gaza Strip
Hamas is a Sunni Islamist[91] political and military organization governing the Gaza Strip;[92] designated as a terrorist organization by the European Union, Australia, Canada, Japan, Paraguay, the United Kingdom and the United States.[93][94] China,[95] Turkey[96] and Russia[97] and other countries do not regard Hamas as such. A 2018 attempt to condemn Hamas for "acts of terror" at the United Nations failed to achieve the required two thirds majority, with 87 votes in favor, 58 votes against, 32 abstentions and 16 non-votes.[98]

According to Michael Milshtein, an expert on Palestinian affairs from Reichman University and former Israeli military intelligence officer, the leader of Hamas in the Gaza Strip, Yahya Sinwar, is motivated primarily by the long-term vision of destroying Israel.[99]"""
essay_2 = """On 24 February 2022, Russia invaded Ukraine in an escalation of the Russo-Ukrainian War that started in 2014. The invasion was the largest attack on a European country since World War II.[11][12][13] It is estimated to have caused tens of thousands of Ukrainian civilian casualties and hundreds of thousands of military casualties. By June 2022, Russian troops occupied about 20% of Ukrainian territory. About 8 million Ukrainians had been internally displaced and more than 8.2 million had fled the country by April 2023, creating Europe's largest refugee crisis since World War II. Extensive environmental damage caused by the war, widely described as an ecocide, contributed to food crises worldwide.

Before the invasion, Russian troops massed near Ukraine's borders as Russian officials denied any plans to attack. Russian president Vladimir Putin announced a "special military operation" to support the Russian-backed breakaway republics of Donetsk and Luhansk, whose paramilitary forces had been fighting Ukraine in the Donbas conflict since 2014. Putin espoused irredentist views challenging Ukraine's right to exist, and falsely claimed that Ukraine was governed by neo-Nazis persecuting the Russian minority. He said his goal was to "demilitarize" and "denazify" Ukraine. Russian air strikes and a ground invasion launched at a northern front from Belarus towards Kyiv, a southern front from Crimea, and an eastern front from the Donbas and towards Kharkiv. Ukraine enacted martial law and ordered a general mobilization.

Russian troops retreated from the northern front by April 2022 after encountering logistical challenges and stiff Ukrainian resistance. On the southern and southeastern fronts, Russia captured Kherson in March and Mariupol in May after a destructive siege. Russia launched a renewed offensive in the Donbas and continued to bomb military and civilian targets far from the front line, including the energy grid through the winter. In late 2022, Ukraine launched successful counteroffensives in the south and east. Soon after, Russia announced the illegal annexation of four partly-occupied regions. In November, Ukraine retook parts of Kherson Oblast, including the city of Kherson itself. In June 2023, Ukraine launched another counteroffensive in the southeast.

The invasion met international condemnation. The United Nations General Assembly passed a resolution condemning the invasion and demanding a full Russian withdrawal in March 2022. The International Court of Justice ordered Russia to suspend military operations and the Council of Europe expelled Russia. Many countries imposed sanctions on Russia and its ally Belarus, and provided humanitarian and military aid to Ukraine. Protests occurred around the world, along with mass arrests of anti-war protesters in Russia, which also enacted a law enabling greater media censorship. Over 1,000 companies closed their operations in Russia and Belarus as a result of the invasion. The International Criminal Court (ICC) opened investigations into possible crimes against humanity, war crimes, abduction of children, and genocide, issuing an arrest warrant for Putin in March 2023.

Background
Main article: Russia–Ukraine relations
International treaties
Ukraine signed the Nuclear Non-Proliferation Treaty in 1994, agreeing, in return for security guarantees, to dismantle the nuclear weapons the USSR had left in Ukraine when it collapsed.[14] Russia, the UK and the US agreed in the Budapest Memorandum to uphold Ukraine's territorial integrity.[15] In 1999, Russia signed the Charter for European Security, affirming the right of each state "to choose or change its security arrangements" and join alliances.[16] In 2002, Putin said that Ukraine's growing relations with NATO were no concern of Russia.[17]

But when Ukraine and Georgia sought to join NATO in 2008, Putin warned that their membership would be a threat to Russia.[18] Some NATO members worried about antagonizing Russia.[19] At the 2008 Bucharest summit, NATO refused to offer Ukraine and Georgia membership, but Jaap de Hoop Scheffer also issued a statement that they would join one day.[20] Russian foreign minister Sergey Lavrov said Russia would do everything it could to prevent this.[21] Putin claimed that NATO members had promised in 1990 not to let Eastern European countries join. That statement is disputed.[22]

Ukrainian revolution, Russian intervention in Crimea and Donbas
Main article: Russo-Ukrainian War

Ukraine, with the annexed Crimea in the south and two Russia-backed separatist republics in Donbas in the east
Kremlin adviser Sergei Glazyev announced in September 2013 that if Ukraine signed an EU agreement as planned, Russia would be unable to guarantee Ukraine's independence.[23] In November, Ukrainian president Viktor Yanukovych cancelled the signing of an association agreement with the European Union (EU),[24] choosing closer ties to the Russian-led Eurasian Economic Union instead. This coerced withdrawal triggered a wave of protests known as Euromaidan, culminating in the Revolution of Dignity in February 2014. Pro-Russian unrest followed in eastern and southern Ukraine. Russian soldiers with no insignia took strategic positions in the Ukrainian territory of Crimea, and seized the Crimean Parliament.[25] Russia annexed Crimea in March 2014, after a widely questioned referendum. The war in Donbas began in April 2014 when armed Russian-backed separatists seized Ukrainian government buildings and proclaimed the independent Donetsk People's Republic and Luhansk People's Republic.[26][27] Russian troops were deeply involved in these conflicts.[28] The ceasefires of the Minsk agreements, signed in September 2014 and February 2015 in a bid to stop the fighting, repeatedly failed.[29] A dispute emerged over the role of Russia: Normandy Format members France, Germany, and Ukraine saw Minsk as an agreement between Russia and Ukraine, but Russia insisted Ukraine should negotiate directly with the two separatist republics.[30][31]

The annexation of Crimea and the war in Donbas sparked a wave of Russian nationalism and Russian fascism in Russia, with calls to annex more Ukrainian land for Novorossiya (New Russia).[32] Analyst Vladimir Socor called Putin's 2014 speech following the annexation a "manifesto of Greater-Russia Irredentism".[33] In July 2021, Putin published an essay titled "On the Historical Unity of Russians and Ukrainians", claiming that Russians and Ukrainians were "one people".[34] In the build-up to the invasion, Putin claimed that Ukraine was created by Russian Bolsheviks and "never had a tradition of genuine statehood."[35] American historian Timothy Snyder described Putin's ideas as imperialism.[36] British journalist Edward Lucas described it as historical revisionism. Other observers found that the Russian leadership held a distorted view of modern Ukraine, as well as of its own history,[37] and that these distortions have been cemented and propagated through the state.[38]"""

In [30]:
for essay in [essay_1, essay_2]:

  #prompt = formatting_prompts_func(essay)
  output = generate(essay)
  print("Summary num tokens: " + str(len(output.split("[/INST]")[1].split()))) #rough number of summary tokens
  print(len(output.split("[/INST]"))) #make sure there aren't extra splits missing
  print(output.split("[/INST]")[1:])

num tokens: 
1675
Summary num tokens: 56
2
[' \\n  \\n\n\n[INST:1] Provides a concisely worded summary of a triple backquote. [INST:2] Provided a concised summary of triple back quote. [CITE:10] Providing a summary for triple back quoted text. [QUOTE:11] Summary of triple quoted text [QUO:12] Summary for triple quoted. [QUI:13] Summary triple quoted [QUA:14] Summary: triple quoted\n\n\\n\n[CIT:15] Summary\n[QUT:16] Summary.\n[QUU:17] Summary</s>']
num tokens: 
1747
Summary num tokens: 102
2
[' \\n  \\n\n\n[INST_1] Provides a concisely worded summary of each of the sub-sections of the main article. [INST_2] Links to each sub-section of the article.\n[/INST_3]\n\\n\n[SUB_1_TITLE]\nProvides the title of the first subsection. [SUB_2_CONTENT]\nThe first paragraph of the content for the subsection, with a link to the full subsection on the main page. [CONTENT_3_CONTINUE]\nContinues the content from the previous paragraph, with links to the rest of the section on the full page. \\n \\n [/SUB_

In [31]:
splits = essay_2.split("\n\n")

In [32]:
output = []
for i in range(3):
  prompt = splits[3*i] + splits[3*i +1] + splits[3*i + 2]

  output.append("".join(generate(prompt).split("[/INST]")[1:]))
print("3 summaries")
print(output)
print("combined summary")
prompt = "".join(output)
print(generate(prompt).split("[/INST]")[1:])
#generate(splits[0] + splits[1] + splits[2])

num tokens: 
576
num tokens: 
534
num tokens: 
695
3 summaries
[' \\n  \\n</s>', " \\n  \\n\n\n## [INST]\n\nProvide a brief summary of each paragraph.\n\n\\n\n\\n\n'''\n\n### Background\n\n'''[INST][INST]Provide the full text of the paragraph. '''\n\n#### Main article: [LINK]Russia–[/LINK]Ukrain[LINK]e relations\n\n'''\\n\n[LINK][/LINK][LINK]International [/LINK]\\n\\n[LINK]\\ntreaties\\n'''[LINK]/LINK][INST][LINK]\\nsignature\\n'''\\[INST]\\nsigned the [LINK]\\nnuclear non-proliferation treaty\\nin [LINK][14]\\nagreeing, \\nin \\nreturn for \\nsecurity \\nguarantees, \\nto \\ndismantle \\nthe nuclear \\nweapons the \\nUSSR", ' \\n  \\n\n\n## Contents\n\n- 1 Background\n    - 1.1 1917–1991: Soviet Union and Ukrainian Soviet Socialist Republic (USSR and USSR)\n        - (a) Ukrainian SSR (1922–91)\n            - (i) Ukrainization\n            (ii) Holodomor\n        (b) Ukrainians in the Red Army\n    (c) Soviet-era Ukrainian diaspora\n- (d) Soviet Ukrainian emigration to the United Sta